# Create spreadsheet with Supersite data for Chairs and Candidates  

Create a spreadsheet that combines Supersite data from several sources into views designed for Supersite Chairs and Candidates.

- pctss dataframe 
    - INPUT: pct_supersite.xlsx
    - each row has precinct information including 2024 supersite
- Create ss_areas dataframe
    - each row has Supersite data aggregated from pctss
    - group by pctss by supersite
    - create columns with lists of all the Areas, SDs, HDs at each Supersite
- Create hd_ss 
    - each row has hd data
    - group pctss by hd
    - create columns with lists of Supersites and precincts for each hd
- Create sd_ss 
    - each row has sd data
    - group pctss by sd
    - create columns with lists of Supersites and precincts for each sd

- write Excel spreadsheet with information for Supersite Chairs and candidates
    - OUTPUT: supersite_area_HD_SD_DRAFT_2024.xlsx
        - write ss_areas to one worksheet
        - write sd_ss to one worksheet
        - write hd_ss to one worksheet
        
## Possible enhancements
- DONE: Transform Areas column from Python list to string, eg. [BO-03, BO-04, MT-03] to "BO-03, BO-04, MT-03"
- Transform Areas column from AreasShort to AreasLong format e.g. "Boulder 03"

- DONE: create list of Supersites for each SD, HD

In [ ]:
import pandas as pd

In [ ]:
# read in pct_supersite.xlsx

cols = ['Pct', 'AreaShort', 'SD', 'HD', 'supersite']
pctss = pd.read_excel('data/pct_supersite.xlsx', usecols=cols)

pctss.info()
pctss

In [ ]:
# create ss_areas dataframe with columns of lists of Areas, SDs and HDs at each Supersite

def collist(s):
    return s.unique().tolist()

ss_areas = pctss.groupby('supersite').agg(
    areas=('AreaShort', collist),
    SDs=('SD', collist),
    HDs=('HD', collist),
    PCTs=('Pct', collist)
)

# convert lists in areas, SDs, HDs, PCTs to strings

ss_areas['area'] = [','.join(map(str, l)) for l in ss_areas['areas']]
ss_areas['sd'] = [','.join(map(str, l)) for l in ss_areas['SDs']]
ss_areas['hd'] = [','.join(map(str, l)) for l in ss_areas['HDs']]
ss_areas['pct'] = [','.join(map(str, l)) for l in ss_areas['PCTs']]

ss_areas = ss_areas[['area', 'sd', 'hd', 'pct']]


ss_areas.info()
ss_areas

## Create list of Supersites by SD, HD
- SD18 at Niwot, New Vista, Monarch, Eldorado K8, Centaurus, ...

In [ ]:
# Create dataframe that lists which supersites and precincts are part of each House District

hd_ss = pctss.groupby('HD').agg(
    supersites=('supersite', collist),
    PCTs=('Pct', collist)
)


In [ ]:
# Create dataframe that lists which supersites and precincts are part of each Senate District
sd_ss = pctss.groupby('SD').agg(
    supersites=('supersite', collist),
    PCTs=('Pct', collist)
)


## Write Supersite data, HD data and SD data to Excel spreadsheet

In [ ]:
# Write excel spreadsheet with separate tabs for ss_areas, hd_ss, sd_ss

with pd.ExcelWriter('data/supersite_area_HD_SD_DRAFT_2024.xlsx') as writer:
    
    ss_areas.to_excel(writer, sheet_name='Areas, SD, HD at Supersite', index=True)
    hd_ss.to_excel(writer, sheet_name="HDs")  
    sd_ss.to_excel(writer, sheet_name="SDs")  